In [1]:
import os
import csv
import pandas as pd

from dotenv import load_dotenv
from github import Github

In [2]:
# Load environment variables
auth_token = os.getenv("GITHUB_TOKEN")
username = os.getenv("GITHUB_USERNAME")

# Using an access token
g = Github(auth_token)
 
# Get the repositories
repos = g.get_repos()
users = []

In [3]:
num_repos = 1000
repos = list(repos[:num_repos])
repos_dataset = []
# repos = ["r_"+str(i)]

In [4]:
users = []
for i, repo in enumerate(repos):
    # s = repo.get_subscribers()
    try:
        r_id = "r_" + str(i)
        languages = list(repo.get_languages().keys())
        full_name = repo.full_name
        repos_dataset.append({'id': r_id, 'name': full_name, 'languages': languages})
        
        subs = repo.get_subscribers()
        subs = [{'repos': r_id, 'user': user.login} for user in subs]        
        users.extend(list(subs))
    except Exception as e:
        print('Error for repo: ', repo.full_name, e)
        continue

Error for repo:  ivey/merb-for-rails 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/reference/repos#list-repository-languages"}
Error for repo:  reinh/dm 403 {"message": "Repository access blocked", "block": {"reason": "unavailable", "created_at": "2014-01-31T22:32:14Z", "html_url": "https://github.com/tos"}}
Error for repo:  chriseppstein/ruby-freebase 403 {"message": "Repository access blocked", "block": {"reason": "unavailable", "created_at": "2014-01-31T22:32:14Z", "html_url": "https://github.com/tos"}}
Error for repo:  efrendiaz/helloworld 403 {"message": "Repository access blocked", "block": {"reason": "unavailable", "created_at": "2014-01-31T22:32:14Z", "html_url": "https://github.com/tos"}}
Error for repo:  soylentfoo/atlrug-demo 403 {"message": "Repository access blocked", "block": {"reason": "unavailable", "created_at": "2014-01-31T22:32:14Z", "html_url": "https://github.com/tos"}}
Error for repo:  mvanholstyn/mhs_testing 500 null


In [5]:
# reduce the users by username
users_df = pd.DataFrame(users)

users_df = users_df.groupby('user')['repos'].apply(list).reset_index(name='repos')
users_id = ["u_"+str(i) for i in range(len(users_df))]
users_df['id'] = users_id
users_df = users_df[['id', 'user', 'repos']]

users_df.to_csv('dataset/users.csv', index=False)

In [6]:
pd.DataFrame(repos_dataset).to_csv('dataset/repos.csv', index=False)